### Part 2

For this portion of the lab, we are going to alter the script, M10_assignment.ipynb, to iterate through the pagination that exists on the page and compile all of the results in one dataframe from each page. Once done , we will save the csv into s3 bucket.

In [ ]:
###Load modules
#!pip install webdriver-manager
import awscli
import boto3
import selenium
import pandas as pd
import time
from io import StringIO
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from botocore.exceptions import ClientError

####SCRAPE THE WEBSITE######
###call the webdriver
s = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s)

#enter the url path that needs to be accessed by webdriver
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities.jsp')

#identify xpath of location to select element
inputElement = browser.find_element(
    By.XPATH, "/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
inputElement.send_keys('0')
inputElement1 = browser.find_element(
    By.XPATH, "/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()
sleep(4)  # allow for the page to load by adding a sleep element
#identify the table to scrape
table = browser.find_element(By.CSS_SELECTOR, 'table.Bordered')
sleep(1)


In [ ]:
# Create empty dataframe
data_list = []

# Loop through pages of results

while True:
    # Loop through dataframe to export table
    for row in table.find_elements(By.CSS_SELECTOR, 'tr'):
        cols = [cell.text for cell in row.find_elements(By.CSS_SELECTOR, 'td')]
        if len(cols) > 0:  # exclude empty rows
            data_list.append({"Organization Name": cols[0], "NY Reg #": cols[1], "EIN": cols[2],
                              "Registrant Type": cols[3], "City": cols[4], "State": cols[5]})

    # Check if there is another page of results
    next_button = browser.find_elements(
        By.XPATH, "//a[contains(text(),'Next')]")
    if not next_button:
        break
    next_button[0].click()
    time.sleep(2)  # allow for the page to load by adding a sleep element
    table = browser.find_element(By.CSS_SELECTOR, 'table.Bordered')

# Concatenate all the dataframes into a single dataframe
final_data = pd.concat([pd.DataFrame(list_num, index=[0])
               for list_num in data_list], ignore_index=True)

# Display the scraped data
display(final_data)


### Creating bucket

We can create bucket using aws console or using boto3 module in python.
In order to create s3 bucket using boto3, we need access key and secret key to access our aws account.

In [ ]:
# # Set the name of the bucket to create
bucket_name = 'm10-assignment'

aws_access_key = input("enter your access key \n")
aws_secret_key = input("enter your secret key \n")

aws_s3_client = boto3.client('s3', aws_access_key_id=aws_access_key,
                             aws_secret_access_key=aws_secret_key)

#Below commented code has the access key to my AWS account.
response = aws_s3_client.list_buckets()
bucket_exist = False

for bucket in response['Buckets']:
    if bucket['Name'] == bucket_name:
        bucket_exist = True
        break

if bucket_exist:
    print("The bucket exists")
else:
    print("The bucket does not exist")

# Create the bucket if it doesn't exist
if not bucket_exist:
    try:
        aws_s3_client.create_bucket(Bucket=bucket_name)
        print(f"{bucket_name} bucket has been created on AWS S3")
    except ClientError as e:
        print(e)
        print(f"{bucket_name} cannot be created on S3")


### Uploading file in s3 bucket

In [ ]:
pathname = 's3://m10-assignment/'
filename = 'charities_bureau_scrape_group6_part2'  # name of your group
datetime = time.strftime("%Y-%m-%d-%H-%M-%S")  # timestamp
# name of the filepath and csv file
filenames3 = f"{filename}-{datetime}.csv"

def upload_s3(final_data, file_name):
    csv_buffer = StringIO()
    final_data.to_csv(csv_buffer, header=True, line_terminator='\n')
    csv_buffer.seek(0)
    aws_s3_client.put_object(
        Bucket=bucket_name, Body=csv_buffer.getvalue(), Key=file_name, ACL='public-read')
    
upload_s3(final_data, filenames3)
print("Data uploaded successfully ")

#print success message
print(f"Successfully uploaded file to location:{pathname}{filenames3}")

In [ ]:
#link to file
https://m10-assignment.s3.amazonaws.com/charities_bureau_scrape_group6_part2-2023-04-12-20-35-08.csv